# Motor Collisions and Traffic Volumes Data Pipeline
This file takes you through the pipeline the motor collision and traffic volume data goes through in order to ultimately create a master collision-volume dataframe from which the main analysis of this project will be conducted. 

A summary of the files and databases used in this analysis is provided below:

**Database #1: Motor Vehicle Collisions involving Killed or Seriously Injured Persons**

*File 1: Motor Vehicle Collisions with KSI Data (GeoJSON)*
This dataset includes all traffic collisions events where a person was either Killed or Seriously Injured (KSI) from 2006 – 2021. Each collision in this dataset includes information on the date, time, location, cause and circumstances surrounding the collision. 

**Database #2: Traffic Volumes at Intersections for All Modes**
    
*File 2: locations (CSV)*

This dataset contains the coordinates and other geospatial data about each intersection the database has volume counts for.


*File 3: raw-data-2000-2009 (CSV)*

*File 4: raw-data-2010-2019 (CSV)*

*File 5: raw-data-2020-2029 (CSV)*

These datasets contains the turning movement counts observed at specific intersections at specific dates and times, segmented by years 2000-2009, 2010-2019, and 2020-2029. Included are the total volumes observed at specific intersections, segmented by direction of approach, turning movement (if applicable), and mode (car, truck, bus, pedestrian, cyclist, other).


# 1: Downloading the Data

The following section looks to download and convert the various files (containing data on motor collisions, traffic counts, and intersection information) required for this study into dataframes for data cleaning and analysis. 

In [250]:
# Import libraries
import os
import json
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import requests
import geopandas as gpd
import folium
from folium import Marker
from geopy.distance import geodesic
from scipy.spatial import KDTree
from scipy.spatial import cKDTree

### 1.1 Downloading Motor Collision Data
Pulling the motor collision data from the City of Toronto Open Data Portal and converting the json file into a pandas dataframe.

In [223]:
# Following code pulls dictionary (.json file) on motor collisions
motor_collisions = (requests.
                    get('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/0b6d3a00-7de1-440b-b47c-7252fd13929f/resource/355d4464-eb3c-4780-af79-43dd533ae906/download/Motor%20Vehicle%20Collisions%20with%20KSI%20Data.geojson').
                    json()
                   )

In [224]:
# Initialize an empty list to store flattened dictionaries
flat_motor_collisions = []

# Iterate through the 'features' list in the JSON motor_collisions
for feature in motor_collisions['features']:
    # properties entail the characteristics of the crash
    properties = feature['properties']
    # geometry entails the location
    geometry = feature['geometry']
    flat_properties = properties.copy()
    flat_properties.update(geometry)
    flat_motor_collisions.append(flat_properties)
    
# Create a motor_collisions DataFrame from the flattened motor_collisions
motor_collisions_df = pd.DataFrame(flat_motor_collisions)

In [225]:
motor_collisions_df.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates
0,1,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,None,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]"
1,2,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,None,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]"
2,3,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,Yes,None,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]"
3,4,893184,2006,2006-01-01,236,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,...,None,Yes,None,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,MultiPoint,"[[-79.318797, 43.699595]]"
4,5,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,Yes,None,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]"


### 1.2 Downloading Traffic and Intersection data
Pulling the traffic count and intersection data from the City of Toronto Open Data Portal and converts the csv files into pandas dataframes.

In [226]:
# Data that includes strictly geospatial information on the intersections where counts are taken from
intersections = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/b6b2cb6f-0528-48ef-b605-4b56e4ce49d6/download/locations.csv')

# Data on historical intersection volumes from 2000 to 2009
vol_00to09 = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/bb7554d9-cd5d-4fad-aa5b-97339a9018df/download/raw-data-2000-2009.csv')

# Data on historical intersection volumes from 2010 to 2019
vol_10to19 = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/1f60c668-bb8e-4e1e-ac72-3c6558a03fea/download/raw-data-2010-2019.csv')

# Data on historical intersection volumes from 2020 to 2022
vol_20to29 = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/71f08804-46ce-4a92-9e8f-9b0e67927ca6/download/raw-data-2020-2029.csv')

In [227]:
intersections.head()

,_id,location_id,location,lng,lat,centreline_type,centreline_id,px,latest_count_date
0,1,1146,ELLESMERE RD AT PARKINGTON CRES,-79.246254,43.773319,2.0,13446642.0,2296.0,2022-12-17
1,2,1981,YORK MILLS RD E/B & W/B TO DON VALLEY PKWY N/B,-79.334658,43.757336,1.0,440171.0,NaN,2004-04-08
2,3,3468,PARKSIDE DR N/B S OF SPRING RD,-79.454442,43.640512,1.0,30010748.0,NaN,2021-12-15
3,4,3925,RIPLEY AVE AT SOUTH KINGSWAY,-79.475274,43.636780,2.0,13468657.0,NaN,2022-01-27
4,5,3926,BLOOR ST AT SOUTH KINGSWAY & RIVERVIEW GARDENS...,-79.485752,43.648312,2.0,13467247.0,334.0,2022-05-10


In [228]:
vol_00to09.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,12.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,10.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [229]:
vol_00to09.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,12.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,10.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [230]:
vol_00to09.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,12.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,10.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


## 2: Converting Dataframes into GeoDataframes
The following section converts each of the pandas dataframes created in section 1 into geodataframes for geospatial analysis.

### 2.1: Creating the GeoDataframes
This conversion is done by acquiring the longitude and latitude of each dataframe and creating point geometry columns using geopandas. Every single dataframe has a lon and lat with exception to the motor_collisions_df which has a geometry column (though this does not function as a geometry column), and as a result, some work has to be done to create the lon and latitude columns.

In [231]:
# The geometry column of the motor collisionss df contains a set of arrays embedded in each other. As a result, by
# repeatedly indexing the array, you can eventually extract the value itself (hence the 2 [0] followed by a [0] or [1]).
motor_collisions_df['lon'] = motor_collisions_df['coordinates'].apply(lambda x: x[0][0])
motor_collisions_df['lat'] = motor_collisions_df['coordinates'].apply(lambda x: x[0][1])
motor_collisions_df.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates,lon,lat
0,1,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345
1,2,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345
2,3,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943
3,4,893184,2006,2006-01-01,236,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,...,None,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,MultiPoint,"[[-79.318797, 43.699595]]",-79.318797,43.699595
4,5,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943


In [232]:
# We turn the df into a gdf so that it has geometry.
motor_collisions_gdf = gpd.GeoDataFrame(motor_collisions_df, 
                                          geometry = gpd.points_from_xy(motor_collisions_df['lon'], 
                                                                        motor_collisions_df['lat'])
                                       )
motor_collisions_gdf.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates,lon,lat,geometry
0,1,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345,POINT (-79.45249 43.65635)
1,2,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345,POINT (-79.45249 43.65635)
2,3,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943,POINT (-79.19979 43.80194)
3,4,893184,2006,2006-01-01,236,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,...,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,MultiPoint,"[[-79.318797, 43.699595]]",-79.318797,43.699595,POINT (-79.31880 43.69960)
4,5,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943,POINT (-79.19979 43.80194)


In [233]:
# Similarly, converting the intersections and traffic dfs into gdfs so that they have geometry.
intersections_gdf = gpd.GeoDataFrame(intersections, 
                                          geometry = gpd.points_from_xy(intersections['lng'], 
                                                                        intersections['lat'])
                                       )

vol_00to09_gdf = gpd.GeoDataFrame(vol_00to09, 
                                          geometry = gpd.points_from_xy(vol_00to09['lng'], 
                                                                        vol_00to09['lat'])
                                       )
vol_10to19_gdf = gpd.GeoDataFrame(vol_10to19, 
                                          geometry = gpd.points_from_xy(vol_10to19['lng'], 
                                                                        vol_10to19['lat'])
                                       )
vol_20to29_gdf = gpd.GeoDataFrame(vol_20to29, 
                                          geometry = gpd.points_from_xy(vol_20to29['lng'], 
                                                                        vol_20to29['lat'])
                                       )

### 2.2 Assigning a Coordinate Reference System (CRS) to each GeoDataframe
A coordinate reference system (CRS) shows how projected points correspond to real locations on Earth. Each GeoDataFrame's crs is checked using the .crs method.  

In [234]:
# Doing this check finds that there is no coordinate system assigned to any of the gdfs, so we can pick one to assign it.
print(motor_collisions_gdf.crs)
print(intersections_gdf.crs)
print(vol_00to09_gdf.crs)
print(vol_10to19_gdf.crs)
print(vol_20to29_gdf.crs)

None
None
None
None
None


Doing this check finds that there is no coordinate system assigned to any of the gdfs, so we can pick one to assign it. As we're working in a relatively small area (Toronto) and we'll be looking at areas and distances, we transform to a 2D projection crs. The City of Mississauga distributes datasets in UTM NAD83 Zone 17N (EPSG:26917) so we set the crs to this. 

In [235]:
#The crs of the motor_collisions, and intersection and traffic gdfs will be initialized to (EPSG:26917).
motor_collisions_gdf.crs = {'init': 'epsg:26917'}
intersections_gdf.crs = {'init': 'epsg:26917'}
vol_00to09_gdf.crs = {'init': 'epsg:26917'}
vol_10to19_gdf.crs = {'init': 'epsg:26917'}
vol_20to29_gdf.crs = {'init': 'epsg:26917'}

C:\Users\princ\anaconda3\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


## 3: Creating a Master Collision-Volume GeoDataframe
The following section looks to create a master dataframe that merges the traffic count volumes data (stored in vol_00to09_gdf, vol_10to19_gdf, and vol_20to29_gdf) corresponding to each collision in the motor_collisions_gdf. This will allow each collision to be contexualized in terms of the traffic volumes and accident frequency at intersections.

### 3.1: Aggregating the Volume GeoDataframes
A summative traffic volume count geodataframe will be created to aggregate all the historical data from vol_20to29_gdf, vol_10to19_gdf, and vol_00to09_gdf.

In [236]:
# Combine all volume GeoDataFrames into one
volumes_gdf = pd.concat([vol_20to29_gdf, vol_10to19_gdf, vol_00to09_gdf], ignore_index=True)

# Print the combined GeoDataFrame
volumes_gdf.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other,geometry
0,1,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
1,2,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
2,3,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
3,4,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
4,5,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)


### 3.2: Filtering for the Target Analysis Years
This motor collisions analysis defined an analysis period of 11 years (2011-2022) of the available data such that sufficient data is included before, during, and after the COVID pandemic. Both the motor_collisions_gdf and the volumes_gdf will be filtered to only include data from 2011-2022.

In [237]:
# Keep the years 2011-2022
# Filter motor_collisions_gdf
motor_collisions_gdf = motor_collisions_gdf[(motor_collisions_gdf['YEAR'] >= 2011) & (motor_collisions_gdf['YEAR'] <= 2022)]

# Filter volumes_gdf
volumes_gdf = volumes_gdf[(volumes_gdf['count_date'] >= '2011-01-01') & (volumes_gdf['count_date'] <= '2022-12-31')]

### 3.3: Converting Date and Time Columns to Datetime Objects
In order to conduct temporal analysis on the two GeoDataframes and eventually merge them, the dates and times of the collisions and traffic volume counts need to be consistently formatted, this is achieved by turning them into datetime objects. 

In [238]:
# Create a copy of motor_collisions_gdf
motor_collisions_gdf_copy = motor_collisions_gdf.copy()

# Pad the TIME values with zeros and convert to a valid time format (HH:MM)
motor_collisions_gdf_copy['TIME'] = motor_collisions_gdf_copy['TIME'].apply(lambda x: x.zfill(4))
motor_collisions_gdf_copy['datetime'] = pd.to_datetime(motor_collisions_gdf_copy['DATE'] + ' ' + motor_collisions_gdf_copy['TIME'].str[:2] + ':' + motor_collisions_gdf_copy['TIME'].str[2:])

# Display the updated DataFrame
motor_collisions_gdf_copy[['DATE', 'TIME', 'datetime']].head()

,DATE,TIME,datetime
6000,2011-03-09,0726,2011-03-09 07:26:00
6001,2011-03-09,1145,2011-03-09 11:45:00
6002,2011-03-09,1145,2011-03-09 11:45:00
6003,2011-03-09,1145,2011-03-09 11:45:00
6004,2011-03-12,1917,2011-03-12 19:17:00


In [239]:
# Create a copy of volumes_gdf
volumes_gdf_copy = volumes_gdf.copy()

# Add the 'datetime_start' and 'datetime_end' columns
volumes_gdf_copy['datetime_start'] = pd.to_datetime(volumes_gdf_copy['time_start'])
volumes_gdf_copy['datetime_end'] = pd.to_datetime(volumes_gdf_copy['time_end'])

# Display the updated DataFrame
volumes_gdf_copy[['count_date', 'time_start', 'datetime_start', 'time_end', 'datetime_end']].head()

,count_date,time_start,datetime_start,time_end,datetime_end
0,2020-01-08,2020-01-08T07:30:00,2020-01-08 07:30:00,2020-01-08T07:45:00,2020-01-08 07:45:00
1,2020-01-08,2020-01-08T07:45:00,2020-01-08 07:45:00,2020-01-08T08:00:00,2020-01-08 08:00:00
2,2020-01-08,2020-01-08T08:00:00,2020-01-08 08:00:00,2020-01-08T08:15:00,2020-01-08 08:15:00
3,2020-01-08,2020-01-08T08:15:00,2020-01-08 08:15:00,2020-01-08T08:30:00,2020-01-08 08:30:00
4,2020-01-08,2020-01-08T08:30:00,2020-01-08 08:30:00,2020-01-08T08:45:00,2020-01-08 08:45:00


### 3.4: Creating the Daily and Hourly Volumes Dataframes
The volumes_gdf is comprised of data for a single day at a single location. A TMC typically includes data for a total of 8 non-continuous hours throughout the day, and the data is reported in a series of 15-minute intervals. Analysis for traffic flow is typically done on an hourly and daily basis. The goal is to merge the relevant hourly and daily volume data for each location and time a collision took place. To achieve this, a daily_volumes_gdf and hourly_volumes_gdf will be created to merge with motor_collisions_gdf.

In [240]:
# Specify the columns to sum
columns_to_sum = ['sb_cars_r', 'sb_cars_t', 'sb_cars_l', 'nb_cars_r',
                  'nb_cars_t', 'nb_cars_l', 'wb_cars_r', 'wb_cars_t', 'wb_cars_l',
                  'eb_cars_r', 'eb_cars_t', 'eb_cars_l', 'sb_truck_r', 'sb_truck_t',
                  'sb_truck_l', 'nb_truck_r', 'nb_truck_t', 'nb_truck_l', 'wb_truck_r',
                  'wb_truck_t', 'wb_truck_l', 'eb_truck_r', 'eb_truck_t', 'eb_truck_l',
                  'sb_bus_r', 'sb_bus_t', 'sb_bus_l', 'nb_bus_r', 'nb_bus_t', 'nb_bus_l',
                  'wb_bus_r', 'wb_bus_t', 'wb_bus_l', 'eb_bus_r', 'eb_bus_t', 'eb_bus_l',
                  'nx_peds', 'sx_peds', 'ex_peds', 'wx_peds', 'nx_bike', 'sx_bike',
                  'ex_bike', 'wx_bike', 'nx_other', 'sx_other', 'ex_other', 'wx_other']

# Group by 'count_date' and 'location', and sum the specified columns with 'daily_' prefix
daily_volumes_gdf = volumes_gdf_copy.groupby(['count_date', 'location', 'location_id', 'lng', 'lat', 'geometry'])[columns_to_sum].sum().reset_index()

# Convert 'count_date' to pandas datetime object
daily_volumes_gdf['count_date'] = pd.to_datetime(daily_volumes_gdf['count_date'])

# Rename columns with 'daily_' prefix
daily_volumes_gdf.columns = ['daily_' + col if col in columns_to_sum else col for col in daily_volumes_gdf.columns]

# Display the new DataFrame
daily_volumes_gdf.head()

,count_date,location,location_id,lng,lat,geometry,daily_sb_cars_r,daily_sb_cars_t,daily_sb_cars_l,daily_nb_cars_r,...,daily_ex_peds,daily_wx_peds,daily_nx_bike,daily_sx_bike,daily_ex_bike,daily_wx_bike,daily_nx_other,daily_sx_other,daily_ex_other,daily_wx_other
0,2011-01-04,ASHTONBEE ROAD AT HAKIMI AVE (PX 2069),33555,-79.292071,43.729972,POINT (-79.292 43.730),41.0,16.0,64.0,634.0,...,105.0,50.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
1,2011-01-04,O CONNOR DR AT SUNRISE AVE (PX 520),4414,-79.304468,43.718906,POINT (-79.304 43.719),494.0,4149.0,293.0,344.0,...,276.0,59.0,48.0,0.0,51.0,0.0,0.0,0.0,0.0,0.0
2,2011-01-04,PHARMACY AVE AT PRAIRIE DR & TEESDALE PL (PX 1...,4830,-79.285399,43.696998,POINT (-79.285 43.697),301.0,3080.0,10.0,43.0,...,166.0,270.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0
3,2011-01-05,EASTERN AVE AT KNOX AVE (PX 1265),5521,-79.325270,43.662516,POINT (-79.325 43.663),658.0,75.0,76.0,237.0,...,175.0,55.0,19.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0
4,2011-01-06,DENLOW BLVD AT LESLIE ST & SOUTHWELL DR (PX 737),5980,-79.358519,43.745983,POINT (-79.359 43.746),348.0,8023.0,57.0,113.0,...,43.0,136.0,0.0,9.0,0.0,5.0,0.0,0.0,0.0,0.0


In [241]:
# Specify the columns to sum
columns_to_sum = ['sb_cars_r', 'sb_cars_t', 'sb_cars_l', 'nb_cars_r',
                  'nb_cars_t', 'nb_cars_l', 'wb_cars_r', 'wb_cars_t', 'wb_cars_l',
                  'eb_cars_r', 'eb_cars_t', 'eb_cars_l', 'sb_truck_r', 'sb_truck_t',
                  'sb_truck_l', 'nb_truck_r', 'nb_truck_t', 'nb_truck_l', 'wb_truck_r',
                  'wb_truck_t', 'wb_truck_l', 'eb_truck_r', 'eb_truck_t', 'eb_truck_l',
                  'sb_bus_r', 'sb_bus_t', 'sb_bus_l', 'nb_bus_r', 'nb_bus_t', 'nb_bus_l',
                  'wb_bus_r', 'wb_bus_t', 'wb_bus_l', 'eb_bus_r', 'eb_bus_t', 'eb_bus_l',
                  'nx_peds', 'sx_peds', 'ex_peds', 'wx_peds', 'nx_bike', 'sx_bike',
                  'ex_bike', 'wx_bike', 'nx_other', 'sx_other', 'ex_other', 'wx_other']

# Round datetime objects to the nearest hour
volumes_gdf_copy['hourly_datetime'] = pd.to_datetime(volumes_gdf_copy['datetime_start']).dt.round('H')

# Group by 'hourly_datetime', 'location', and 'location_id', and sum the specified columns with 'hourly_' prefix
hourly_volumes_gdf = volumes_gdf_copy.groupby(['hourly_datetime', 'location', 'location_id', 'lng', 'lat', 'geometry'])[columns_to_sum].sum().reset_index()

# Rename columns with 'hourly_' prefix
hourly_volumes_gdf.columns = ['hourly_' + col if col in columns_to_sum else col for col in hourly_volumes_gdf.columns]

# Display the new DataFrame
hourly_volumes_gdf.head()

,hourly_datetime,location,location_id,lng,lat,geometry,hourly_sb_cars_r,hourly_sb_cars_t,hourly_sb_cars_l,hourly_nb_cars_r,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,2011-01-04 08:00:00,ASHTONBEE ROAD AT HAKIMI AVE (PX 2069),33555,-79.292071,43.729972,POINT (-79.292 43.730),3.0,2.0,10.0,47.0,...,15.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2011-01-04 08:00:00,O CONNOR DR AT SUNRISE AVE (PX 520),4414,-79.304468,43.718906,POINT (-79.304 43.719),128.0,735.0,38.0,48.0,...,41.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2011-01-04 08:00:00,PHARMACY AVE AT PRAIRIE DR & TEESDALE PL (PX 1...,4830,-79.285399,43.696998,POINT (-79.285 43.697),26.0,542.0,3.0,20.0,...,4.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011-01-04 09:00:00,ASHTONBEE ROAD AT HAKIMI AVE (PX 2069),33555,-79.292071,43.729972,POINT (-79.292 43.730),0.0,1.0,3.0,34.0,...,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011-01-04 09:00:00,O CONNOR DR AT SUNRISE AVE (PX 520),4414,-79.304468,43.718906,POINT (-79.304 43.719),72.0,423.0,17.0,33.0,...,28.0,11.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


### 3.5: Finding the Closest Counts for each Collision
In order to join the motor_collisions_gdf with the hourly_volumes_gdf and daily_volumes_gdf, the coordinates of the count intersections (from intersections_gdf) and the collision intersection need to be matched up. In order to do this, the function find_closest_location() was created to find the coordinates of the closest count to the coordinates of the collision.

In [242]:
intersections_gdf_copy = intersections_gdf.copy()

# Create a KDTree for fast nearest-neighbor queries
kdtree = cKDTree(intersections_gdf_copy[['lat', 'lng']])

# Create a function to find the closest location
def find_closest_location(row):
    # Coordinates from motor_collisions_gdf
    motor_coords = (row['lat'], row['lon'])

    # Query the KDTree to find the index of the nearest neighbor
    _, closest_location_index = kdtree.query(motor_coords)

    try:
        # Get the name of the closest location
        closest_location = intersections_gdf_copy.loc[closest_location_index, 'location']
    except KeyError:
        # Handle the case where the index is not present
        closest_location = None

    return closest_location

# Apply the function to create the 'closest_count_location' column
motor_collisions_gdf_copy['closest_count_location'] = motor_collisions_gdf_copy.apply(find_closest_location, axis=1)

# Display the updated DataFrame
motor_collisions_gdf_copy.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location
6000,6001,1237939,2011,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,78,Kensington-Chinatown (78),D14,MultiPoint,"[[-79.40769, 43.656445]]",-79.40769,43.656445,POINT (-79.408 43.656),2011-03-09 07:26:00,BATHURST ST AT COLLEGE ST (PX 300)
6001,6002,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,89,Runnymede-Bloor West Village (89),D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523)
6002,6003,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,89,Runnymede-Bloor West Village (89),D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523)
6003,6004,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,89,Runnymede-Bloor West Village (89),D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523)
6004,6005,1222866,2011,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,75,Church-Yonge Corridor (75),D51,MultiPoint,"[[-79.37789, 43.658245]]",-79.37789,43.658245,POINT (-79.378 43.658),2011-03-12 19:17:00,CHURCH ST AT GOULD ST (PX 993)


There are some closest_count_locations that do not match up with the intersections from the motor_collisions_gdf ('STREET1' and 'STREET2') because counts do not exist for these locations. Some instances of this mismatching results in a count from a neighbouring intersection being associated with the collision location, while other instances result in a completely distant count. Let's identify these problematic intersections in both cases.

In [243]:
# Indexing the first word of each street to get the root name
motor_collisions_gdf_copy['street1_first_word'] = motor_collisions_gdf_copy['STREET1'].str.split().str[0]
motor_collisions_gdf_copy['street2_first_word'] = motor_collisions_gdf_copy['STREET2'].str.split().str[0]

# Filter rows based on whether the root name exists within the closest_count_location
problematic_intersections_far_away = motor_collisions_gdf_copy[
    ~motor_collisions_gdf_copy.apply(
        lambda row: (
            row['closest_count_location'].lower().find(row['street1_first_word'].lower()) != -1 or
            row['closest_count_location'].lower().find(row['street2_first_word'].lower()) != -1
        ),
        axis=1
    )
]

# Display the problematic intersections far away DataFrame
problematic_intersections_far_away.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location,street1_first_word,street2_first_word
6022,6023,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6023,6024,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6024,6025,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6039,6040,1224559,2011,2011-03-24,1520,WESTON RD,BELLEVUE Cres,None,Minor Arterial,Etobicoke York,...,D12,MultiPoint,"[[-79.51539, 43.699845]]",-79.51539,43.699845,POINT (-79.515 43.700),2011-03-24 15:20:00,LAWRENCE AVE AT SOUTH STATION ST,WESTON,BELLEVUE
6040,6041,1224559,2011,2011-03-24,1520,WESTON RD,BELLEVUE Cres,None,Minor Arterial,Etobicoke York,...,D12,MultiPoint,"[[-79.51539, 43.699845]]",-79.51539,43.699845,POINT (-79.515 43.700),2011-03-24 15:20:00,LAWRENCE AVE AT SOUTH STATION ST,WESTON,BELLEVUE


In [244]:
len(problematic_intersections_far_away) / len(motor_collisions_gdf_copy)

0.12346532941379906

12% of the collisions have a closest_count_location that is far away from its actual location. 

In [245]:
# Indexing the first word of each street to get the root name
motor_collisions_gdf_copy['street1_first_word'] = motor_collisions_gdf_copy['STREET1'].str.split().str[0]
motor_collisions_gdf_copy['street2_first_word'] = motor_collisions_gdf_copy['STREET2'].str.split().str[0]

# Filter rows based on whether the root name exists within the closest_count_location
problematic_intersections_all = motor_collisions_gdf_copy[
    ~motor_collisions_gdf_copy.apply(
        lambda row: (
            row['closest_count_location'].lower().find(row['street1_first_word'].lower()) != -1 and
            row['closest_count_location'].lower().find(row['street2_first_word'].lower()) != -1
        ),
        axis=1
    )
]

# Display the problematic intersections all DataFrame
problematic_intersections_all.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location,street1_first_word,street2_first_word
6022,6023,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6023,6024,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6024,6025,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6028,6029,1213866,2011,2011-01-21,0519,DUFFERIN ST,NORTON AVE W,None,Major Arterial,Toronto and East York,...,D13,MultiPoint,"[[-79.44329, 43.678645]]",-79.44329,43.678645,POINT (-79.443 43.679),2011-01-21 05:19:00,DUFFERIN ST AT ST CLAIR AVE W,DUFFERIN,NORTON
6032,6033,1224587,2011,2011-03-20,2215,MARKHAM RD,DUNELM ST,None,Major Arterial,Scarborough,...,D43,MultiPoint,"[[-79.22069, 43.747145]]",-79.22069,43.747145,POINT (-79.221 43.747),2011-03-20 22:15:00,COUGAR CRT AT LUELLA ST & MARKHAM RD (PX 990),MARKHAM,DUNELM


In [246]:
len(problematic_intersections_all) / len(motor_collisions_gdf_copy)

0.29422445097700156

30% of the collisions do not have an exact count. 

#### Solution: Drop only the collisions with mismatched counts that are far away
To preserve 26% amount of the collisions data, only the mismatched locations that have counts that are far away will be dropped. This would mean that all the collisions in the master dataframe have counts that are from the collision location OR similar counts to what would be expected at the location of the collision. 

In [247]:
# motor_collisions_gdf_copy without all the far away problematic intersections
master_df = motor_collisions_gdf_copy[~motor_collisions_gdf_copy.index.isin(problematic_intersections_far_away.index)].copy()

# Display the master_df DataFrame
master_df.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location,street1_first_word,street2_first_word
6000,6001,1237939,2011,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,D14,MultiPoint,"[[-79.40769, 43.656445]]",-79.40769,43.656445,POINT (-79.408 43.656),2011-03-09 07:26:00,BATHURST ST AT COLLEGE ST (PX 300),COLLEGE,BATHURST
6001,6002,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523),JANE,ST
6002,6003,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523),JANE,ST
6003,6004,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523),JANE,ST
6004,6005,1222866,2011,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,D51,MultiPoint,"[[-79.37789, 43.658245]]",-79.37789,43.658245,POINT (-79.378 43.658),2011-03-12 19:17:00,CHURCH ST AT GOULD ST (PX 993),CHURCH,GOULD


### 3.6: Combining the Daily and Hourly Volumes Dataframes with the Motor Collisions Dataframe

First merging the daily volumes to each collision

In [272]:
def merge_daily_volumes(row):
    # Extract location and date information
    location = row['closest_count_location']
    date = row['datetime']  # Use 'datetime' column directly
    
    # Filter daily_volumes_gdf for the exact location match
    location_match = daily_volumes_gdf[daily_volumes_gdf['location'].str.lower() == location.lower()]
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches
    
    # Find the closest date
    location_match['date_diff'] = abs((location_match['count_date'] - pd.to_datetime(date)).dt.total_seconds())
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches after date filtering
    
    closest_date_index = location_match['date_diff'].idxmin()
    
    # Get the corresponding row from daily_volumes_gdf
    closest_row = location_match.loc[closest_date_index]
    
    # Drop the temporary column
    closest_row = closest_row.drop('date_diff')
    
    # Merge the rows and reset the index to None
    merged_row = pd.concat([row, closest_row]).reset_index(drop=True)
    
    return merged_row

# Apply the function to create the merged DataFrame
master_df_merged = master_df.apply(merge_daily_volumes, axis=1)



# Display the updated DataFrame
master_df_merged.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,daily_ex_peds,daily_wx_peds,daily_nx_bike,daily_sx_bike,daily_ex_bike,daily_wx_bike,daily_nx_other,daily_sx_other,daily_ex_other,daily_wx_other
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,1923.0,1190.0,253.0,234.0,797.0,1647.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,210.0,117.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,210.0,117.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,210.0,117.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,1540.0,934.0,567.0,387.0,129.0,156.0,0.0,0.0,0.0,0.0


Then merging the hourly volumes to each collision

In [278]:
def merge_hourly_volumes(row):
    # Extract location and date information
    location = row['closest_count_location']
    date_time = row['datetime']
    
    # Check for NaN values
    if pd.isna(location) or pd.isna(date_time):
        return pd.Series()  # Return an empty Series if there are NaN values
    
    # Filter hourly_volumes_gdf for the exact location match
    location_match = hourly_volumes_gdf[hourly_volumes_gdf['location'].str.lower() == location.lower()]
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches
    
    # Find the closest datetime
    location_match['datetime_diff'] = abs((location_match['hourly_datetime'] - pd.to_datetime(date_time)).dt.total_seconds())
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches after datetime filtering
    
    closest_datetime_index = location_match['datetime_diff'].idxmin()
    
    # Get the corresponding row from hourly_volumes_gdf
    closest_row = location_match.loc[closest_datetime_index]
    
    # Drop the temporary column
    closest_row = closest_row.drop('datetime_diff')
    
    # Merge the rows
    merged_row = pd.concat([row, closest_row])
    
    # Reset the index to default (0, 1, 2, ...)
    merged_row = merged_row.reset_index(drop=True)
    
    return merged_row

# Apply the function to create the merged DataFrame for hourly volumes
master_df_hourly_merged = master_df_merged.apply(merge_hourly_volumes, axis=1)


# Display the updated DataFrame
master_df_hourly_merged.head()

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,164,165,166,167,168
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [279]:
# Set the columns to the desired column names
master_df_hourly_merged.columns = ['_id', 'ACCNUM', 'YEAR', 'DATE', 'TIME', 'STREET1', 'STREET2', 'OFFSET',
       'ROAD_CLASS', 'DISTRICT', 'WARDNUM', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL',
       'VISIBILITY', 'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE',
       'INVAGE', 'INJURY', 'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER',
       'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE',
       'CYCACT', 'CYCCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE',
       'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER',
       'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'HOOD_158',
       'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'DIVISION',
       'type', 'coordinates', 'lon', 'lat', 'geometry', 'datetime',
       'closest_count_location', 'street1_first_word', 'street2_first_word', 'count_date', 'location', 'location_id', 'lng', 'lat', 'geometry',
       'daily_sb_cars_r', 'daily_sb_cars_t', 'daily_sb_cars_l',
       'daily_nb_cars_r', 'daily_nb_cars_t', 'daily_nb_cars_l',
       'daily_wb_cars_r', 'daily_wb_cars_t', 'daily_wb_cars_l',
       'daily_eb_cars_r', 'daily_eb_cars_t', 'daily_eb_cars_l',
       'daily_sb_truck_r', 'daily_sb_truck_t', 'daily_sb_truck_l',
       'daily_nb_truck_r', 'daily_nb_truck_t', 'daily_nb_truck_l',
       'daily_wb_truck_r', 'daily_wb_truck_t', 'daily_wb_truck_l',
       'daily_eb_truck_r', 'daily_eb_truck_t', 'daily_eb_truck_l',
       'daily_sb_bus_r', 'daily_sb_bus_t', 'daily_sb_bus_l', 'daily_nb_bus_r',
       'daily_nb_bus_t', 'daily_nb_bus_l', 'daily_wb_bus_r', 'daily_wb_bus_t',
       'daily_wb_bus_l', 'daily_eb_bus_r', 'daily_eb_bus_t', 'daily_eb_bus_l',
       'daily_nx_peds', 'daily_sx_peds', 'daily_ex_peds', 'daily_wx_peds',
       'daily_nx_bike', 'daily_sx_bike', 'daily_ex_bike', 'daily_wx_bike',
       'daily_nx_other', 'daily_sx_other', 'daily_ex_other', 'daily_wx_other',
        'hourly_datetime', 'location', 'location_id', 'lng', 'lat', 'geometry',
       'hourly_sb_cars_r', 'hourly_sb_cars_t', 'hourly_sb_cars_l',
       'hourly_nb_cars_r', 'hourly_nb_cars_t', 'hourly_nb_cars_l',
       'hourly_wb_cars_r', 'hourly_wb_cars_t', 'hourly_wb_cars_l',
       'hourly_eb_cars_r', 'hourly_eb_cars_t', 'hourly_eb_cars_l',
       'hourly_sb_truck_r', 'hourly_sb_truck_t', 'hourly_sb_truck_l',
       'hourly_nb_truck_r', 'hourly_nb_truck_t', 'hourly_nb_truck_l',
       'hourly_wb_truck_r', 'hourly_wb_truck_t', 'hourly_wb_truck_l',
       'hourly_eb_truck_r', 'hourly_eb_truck_t', 'hourly_eb_truck_l',
       'hourly_sb_bus_r', 'hourly_sb_bus_t', 'hourly_sb_bus_l',
       'hourly_nb_bus_r', 'hourly_nb_bus_t', 'hourly_nb_bus_l',
       'hourly_wb_bus_r', 'hourly_wb_bus_t', 'hourly_wb_bus_l',
       'hourly_eb_bus_r', 'hourly_eb_bus_t', 'hourly_eb_bus_l',
       'hourly_nx_peds', 'hourly_sx_peds', 'hourly_ex_peds', 'hourly_wx_peds',
       'hourly_nx_bike', 'hourly_sx_bike', 'hourly_ex_bike', 'hourly_wx_bike',
       'hourly_nx_other', 'hourly_sx_other', 'hourly_ex_other',
       'hourly_wx_other'                   
                           ]


In [289]:
master_df_hourly_merged = master_df_hourly_merged.dropna(how='all')
master_df_hourly_merged.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [290]:
# Rename the DataFrame
master_gdf = master_df_hourly_merged

# Save the DataFrame to a CSV file in the same folder as the code
master_gdf.to_csv("master_gdf.csv", index=False)